In [4]:
import pycuda.autoinit
import pycuda.driver as drv
import numpy as np
import sys
from time import perf_counter 
# returns the float value of time in seconds. 
#Return the value (in fractional seconds) of a performance counter, i.e. a clock with the highest available resolution to measure a short duration. 

from math import ceil
from pycuda import gpuarray
from pycuda.compiler import SourceModule


MATRIX_SIZE = 5000

a_mat = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)
b_mat = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)
c_mat = a_mat.dot(b_mat)

a_mat_gpu = gpuarray.to_gpu(a_mat)
b_mat_gpu = gpuarray.to_gpu(b_mat)
output_mat_gpu = gpuarray.empty((MATRIX_SIZE, MATRIX_SIZE), np.float32)
#the matrix is represented as an array in the gpu so the indexation changes accordingly
ker = SourceModule("""

                   __global__ void matrix_multiplication_kernel(
                   float *a_mat, float *b_mat, float *output_mat){
                    float a_val, b_val;
                    int tx = blockIdx.x*blockDim.x+threadIdx.x;
                    int ty = blockIdx.y*blockDim.y+threadIdx.y;
                    float pval = 0;
                    if (tx < %(MATRIX_SIZE)s && ty < %(MATRIX_SIZE)s){
                     for (int k=0; k < %(MATRIX_SIZE)s; ++k){
                      a_val = a_mat[tx * %(MATRIX_SIZE)s + k];
                      b_val = b_mat[k * %(MATRIX_SIZE)s + ty];
                      pval += a_val * b_val;
                     }
                     output_mat[tx * %(MATRIX_SIZE)s + ty] = pval; 
                    }
                   }"""%{'MATRIX_SIZE': MATRIX_SIZE})

matrix_multiplu_gpu = ker.get_function("matrix_multiplication_kernel")


start = drv.Event()
end=drv.Event()
#Start Time
start.record()
#The kernel code for which time is to be measured
#is converted to next highest integer value using the ceil function of the numpy library
matrix_multiplu_gpu(a_mat_gpu, b_mat_gpu, output_mat_gpu, block=(32,32,1), grid=(ceil(MATRIX_SIZE/32),ceil(MATRIX_SIZE/32),1))
#End Time
end.record()
end.synchronize()
#Measure time difference, give time in milliseconds, which is converted to seconds.
secs = start.time_till(end)*1e-3
print("Matrix multiplication of %d elements"%MATRIX_SIZE)
print("take %fs" % (secs))



#liberar memoria de forma manual
a_mat_gpu.gpudata.free()
b_mat_gpu.gpudata.free()
output_mat_gpu.gpudata.free()



Matrix multiplication of 5000 elements
take 11.931646s
